# `biothings_client` Python wrapper  

  
 `biothings_client`: python wrapper to access Biothings.api   

`get_client` function: BioThings API creation. We only need to specify the entity type you want a client for as a string.  

In [1]:
from biothings_client import get_client, MyGeneInfo
import pandas as pd

---

## MyGene.info API

In [2]:
gene_client = get_client('gene') 
type(gene_client)

biothings_client.MyGeneInfo

In [3]:
mgi = MyGeneInfo()

A `MyGeneInfo` object (exactly as obtained through the [MyGene.py](https://pypi.org/project/mygene/) package) was created. Making accessible all methods through `mygene`.

`get_gene()`

In [4]:
def get_gene(gene_id):
    gene=gene_client.getgene(gene_id, fields='symbol,name')
    return gene

In [5]:
gene=get_gene("563878")#"FBgn0260795")#ZDB-GENE-070112-1002, ZDB-GENE-041114-199, 1311391 \, 
print(gene)

{'_id': '563878', '_version': 1, 'name': 'protein-O-mannosyltransferase 2', 'symbol': 'pomt2'}


In [6]:
 y=gene_client.getgene('1017', fields='symbol,name')

In [17]:
print(y)

{'_id': '1017', '_version': 1, 'name': 'cyclin dependent kinase 2', 'symbol': 'CDK2'}


`gene_client.query(geneKey)`


In [7]:
y=gene_client.query("ZFIN:ZDB-GENE-070112-1002", fields='symbol,name')


In [8]:
print(y)

{'took': 2, 'total': 1, 'max_score': 9.763228, 'hits': [{'_id': '563878', '_score': 9.763228, 'name': 'protein-O-mannosyltransferase 2', 'symbol': 'pomt2'}]}


In [9]:
print(y["hits"][0]["_id"])

563878


In [10]:
def get_entrezgene(gene_id, gene_client):
    """
      
        Use the biothings_client package to query the mygene.info db for 
        the corresponding entrezgene(NCBI gene) ID for the given gene_id
        gene_id: given id to search on
        gene_client: initialized object for connection to mygene.info 
    """
    #gene_client = get_client('gene') # initialize mygene object
    if "WB:" in gene_id: gene_id.replace("WB:", "WormBase:")
    #gene=gene_client.getgene(gene_id, fields='symbol,name')
    gene=gene_client.query(gene_id, fields='symbol,name') # get search results
    print(gene)
    #return gene;


In [30]:
get_entrezgene("WormBase:WBGene00011502", gene_client)


{'took': 1, 'total': 1, 'max_score': 10.459784, 'hits': [{'_id': '176377', '_score': 10.459784, 'name': 'Vacuolar protein sorting-associated protein 53 homolog', 'symbol': 'vps-53'}]}


---  

## `biothings.utils`

In [8]:
datafile = "/Users/nacosta/Documents/data/ORTHOLOGY-ALLIANCE_COMBINED_51.tsv"

df=pd.read_csv(datafile, sep="\t", engine="python", header=15)

In [9]:
unique_ids=df["Gene1ID"].unique()

In [10]:
ids=[]

for gene_id in unique_ids:
    ids.append(gene_id.split(":")[1])

In [11]:
ids[:10]



['WBGene00011502',
 'ZDB-GENE-041114-199',
 '1311391',
 '1915549',
 '25608',
 'FBgn0031598',
 'ZDB-GENE-070112-1002',
 '1586427',
 '2444430',
 '19743']

In [12]:
mgi.querymany(ids[:10], fields='ensembl.gene,symbol', as_dataframe=True )



querying 1-10...done.
Finished.
6 input query terms found no hit:
	['ZDB-GENE-041114-199', '1311391', '1915549', 'ZDB-GENE-070112-1002', '1586427', '2444430']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


,_id,_score,symbol,ensembl.gene,notfound
query,,,,,
WBGene00011502,176377,17.044899,vps-53,WBGene00011502,NaN
ZDB-GENE-041114-199,NaN,NaN,NaN,NaN,True
1311391,NaN,NaN,NaN,NaN,True
1915549,NaN,NaN,NaN,NaN,True
25608,25608,18.676743,Lep,ENSRNOG00000045797,NaN
FBgn0031598,33640,17.044899,Vps53,FBgn0031598,NaN
ZDB-GENE-070112-1002,NaN,NaN,NaN,NaN,True
1586427,NaN,NaN,NaN,NaN,True
2444430,NaN,NaN,NaN,NaN,True


In [11]:
mgi.querymany(["176377"], fields='ensembl.gene,symbol',returnall=True)



querying 1-1...done.
Finished.


{'out': [{'query': '176377',
   '_id': '176377',
   '_score': 16.979658,
   'ensembl': {'gene': 'WBGene00011502'},
   'symbol': 'vps-53'}],
 'dup': [],
 'missing': []}

In [34]:
for id_ in ids[:5]:
    print("[INFO] querying %s "%id_)
    a=gene_client.query(id_)
    print(a)

[INFO] querying WormBase:WBGene00011502 
{'took': 2, 'total': 1, 'max_score': 10.459784, 'hits': [{'_id': '176377', '_score': 10.459784, 'entrezgene': '176377', 'name': 'Vacuolar protein sorting-associated protein 53 homolog', 'symbol': 'vps-53', 'taxid': 6239}]}
[INFO] querying ZFIN:ZDB-GENE-041114-199 
{'took': 1, 'total': 1, 'max_score': 9.763228, 'hits': [{'_id': '492817', '_score': 9.763228, 'entrezgene': '492817', 'name': 'VPS53 subunit of GARP complex', 'symbol': 'vps53', 'taxid': 7955}]}
[INFO] querying RGD:1311391 
{'took': 0, 'total': 1, 'max_score': 11.265029, 'hits': [{'_id': '287535', '_score': 11.265029, 'entrezgene': '287535', 'name': 'VPS53 subunit of GARP complex', 'symbol': 'Vps53', 'taxid': 10116}]}
[INFO] querying mgi:MGI\:1915549 
{'took': 2, 'total': 1, 'max_score': 14.001472, 'hits': [{'_id': '68299', '_score': 14.001472, 'entrezgene': '68299', 'name': 'VPS53 GARP complex subunit', 'symbol': 'Vps53', 'taxid': 10090}]}
[INFO] querying HGNC:25608 
{'took': 2, 'tota

---

## MyVariant.info API    


In [31]:
variant_client = get_client('variant')

In [32]:
variant_client.query('dbnsfp.genename:BTK', fields='_id')

{'took': 11,
 'total': 5143,
 'max_score': 10.062878,
 'hits': [{'_id': 'chrX:g.100613660T>A', '_score': 10.062878},
  {'_id': 'chrX:g.100613668A>C', '_score': 10.062878},
  {'_id': 'chrX:g.100613668A>G', '_score': 10.062878},
  {'_id': 'chrX:g.100614304C>G', '_score': 10.062878},
  {'_id': 'chrX:g.100614318A>T', '_score': 10.062878},
  {'_id': 'chrX:g.100614330T>A', '_score': 10.062878},
  {'_id': 'chrX:g.100615578A>G', '_score': 10.062878},
  {'_id': 'chrX:g.100615586A>G', '_score': 10.062878},
  {'_id': 'chrX:g.100615588G>C', '_score': 10.062878},
  {'_id': 'chrX:g.100615593T>A', '_score': 10.062878}]}

## MyChem.info API  

In [33]:
chem_client = get_client('chem')

In [34]:
chem_client.getchem('DB00551', fields='drugbank.name')

## MyDisease.info API  

In [35]:
disease_client = get_client('disease')


In [36]:
disease_client.query('diabetes')

{'took': 15,
 'total': 137,
 'max_score': 9.920628,
 'hits': [{'_id': 'MONDO:0005147',
   '_score': 9.920628,
   'ctd': {'chemical_related_to_disease': [{'cas_registry_number': '53-35-0',
      'chemical_name': '6 beta-hydroxycortisol',
      'direct_evidence': 'marker/mechanism',
      'mesh_chemical_id': 'C001380',
      'pubmed': '16616286',
      'source': 'CTD'},
     {'cas_registry_number': '50-71-5',
      'chemical_name': 'Alloxan',
      'direct_evidence': 'marker/mechanism',
      'mesh_chemical_id': 'D000496',
      'pubmed': '22138235',
      'source': 'CTD'},
     {'cas_registry_number': '80-05-7',
      'chemical_name': 'bisphenol A',
      'direct_evidence': 'marker/mechanism',
      'mesh_chemical_id': 'C006780',
      'pubmed': '24189131',
      'source': 'CTD'},
     {'cas_registry_number': '32222-06-3',
      'chemical_name': 'Calcitriol',
      'direct_evidence': 'therapeutic',
      'mesh_chemical_id': 'D002117',
      'pubmed': '11250657',
      'source': 'CTD'},


## t.biothings.io API - Taxnomy  

In [37]:
taxon_client = get_client('taxon')

---